In [ ]:
# Source: https://nni.readthedocs.io/zh/stable/tutorials/hpo_quickstart_tensorflow/model.html?continueFlag=d47ca216a263dbc2b64a99bca96d70cf

In [1]:
%matplotlib inline


# Port TensorFlow Quickstart to NNI
This is a modified version of `TensorFlow quickstart`_.

It can be run directly and will have the exact same result as original version.

Furthermore, it enables the ability of auto tuning with an NNI *experiment*, which will be detailed later.

It is recommended to run this script directly first to verify the environment.

There are 3 key differences from the original version:

1. In `Get optimized hyperparameters`_ part, it receives generated hyperparameters.
2. In `(Optional) Report intermediate results`_ part, it reports per-epoch accuracy metrics.
3. In `Report final result`_ part, it reports final accuracy.



In [2]:
import nni
import tensorflow as tf

## Hyperparameters to be tuned
These are the hyperparameters that will be tuned later.



In [3]:
params = {
    'dense_units': 128,
    'activation_type': 'relu',
    'dropout_rate': 0.2,
    'learning_rate': 0.001,
}

## Get optimized hyperparameters
If run directly, :func:`nni.get_next_parameter` is a no-op and returns an empty dict.
But with an NNI *experiment*, it will receive optimized hyperparameters from tuning algorithm.



In [4]:
optimized_params = nni.get_next_parameter()
params.update(optimized_params)
print(params)

{'dense_units': 128, 'activation_type': 'relu', 'dropout_rate': 0.2, 'learning_rate': 0.001}


c:\python\python38\lib\site-packages\nni\runtime\platform\standalone.py:32: RuntimeWarning: Running trial code without runtime. Please check the tutorial if you are new to NNI: https://nni.readthedocs.io/en/stable/tutorials/hpo_quickstart_pytorch/main.html
  warnings.warn(warning_message, RuntimeWarning)


## Load dataset



In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 7s 1us/step


## Build model with hyperparameters



In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(params['dense_units'], activation=params['activation_type']),
    tf.keras.layers.Dropout(params['dropout_rate']),
    tf.keras.layers.Dense(10)
])

adam = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=adam, loss=loss_fn, metrics=['accuracy'])

## (Optional) Report intermediate results
The callback reports per-epoch accuracy to show learning curve in the web portal.
You can also leverage the metrics for early stopping with :doc:`NNI assessors </hpo/assessors>`.

This part can be safely skipped and the experiment will work fine.



In [7]:
callback = tf.keras.callbacks.LambdaCallback(
    on_epoch_end = lambda epoch, logs: nni.report_intermediate_result(logs['accuracy'])
)

## Train and evluate the model



In [8]:
model.fit(x_train, y_train, epochs=5, verbose=2, callbacks=[callback])
loss, accuracy = model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
[2022-12-10 14:19:08] Intermediate result: 0.9147666692733765  (Index 0)
1875/1875 - 4s - loss: 0.2924 - accuracy: 0.9148 - 4s/epoch - 2ms/step
Epoch 2/5
[2022-12-10 14:19:11] Intermediate result: 0.9584166407585144  (Index 1)
1875/1875 - 3s - loss: 0.1394 - accuracy: 0.9584 - 3s/epoch - 2ms/step
Epoch 3/5
[2022-12-10 14:19:13] Intermediate result: 0.9686999917030334  (Index 2)
1875/1875 - 3s - loss: 0.1047 - accuracy: 0.9687 - 3s/epoch - 2ms/step
Epoch 4/5
[2022-12-10 14:19:16] Intermediate result: 0.9732333421707153  (Index 3)
1875/1875 - 3s - loss: 0.0868 - accuracy: 0.9732 - 3s/epoch - 2ms/step
Epoch 5/5
[2022-12-10 14:19:19] Intermediate result: 0.9767333269119263  (Index 4)
1875/1875 - 3s - loss: 0.0747 - accuracy: 0.9767 - 3s/epoch - 2ms/step
313/313 - 0s - loss: 0.0794 - accuracy: 0.9730 - 484ms/epoch - 2ms/step


## Report final result
Report final accuracy to NNI so the tuning algorithm can suggest better hyperparameters.



In [9]:
nni.report_final_result(accuracy)

[2022-12-10 14:19:23] Final result: 0.9729999899864197
